In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
import datetime
import json, csv
import os.path
import pandas as pd

In [ ]:
service = QiskitRuntimeService()
# service.backends(simulator=False, operational=True)

In [ ]:
backend = service.backend('ibm_nairobi')

In [ ]:
num_of_days = 315
save_path = '/Users/home/Desktop/college/fa23/research/src/notebooks/data/nairobi'
for i in range(num_of_days):
    date = datetime.datetime(2023, 1, 1, 12) + datetime.timedelta(i)
    properties = backend.properties(datetime=date)
    complete_name = os.path.join(save_path, 'unformatted.json')
    with open(complete_name, 'w', newline='') as file:
        file.write(json.dumps(properties.to_dict(),default=str))
    with open(complete_name) as file:
        unformatted_data = json.load(file)
        file.close()
    dateString = date.strftime('%Y-%m-%d')
    complete_name = os.path.join(save_path, dateString+'-qubits.csv')
    with open(complete_name, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Qubit", "Name", "Unit", "Value"])
        
        for qubit_index, qubits in enumerate(unformatted_data['qubits']):
            for entry in qubits:
                if entry['name'] == 'readout_error':
                    continue
                name = entry['name']
                if entry['unit'] == '':
                    unit = 'per'
                else:
                    unit = entry['unit']
                value = entry['value']
                writer.writerow([qubit_index,name,unit,value])
    
    complete_name = os.path.join(save_path, dateString+'-gates.csv')
    with open(complete_name, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Qubit", "Gate", "Parameter", "Unit", "Value", "Gate Name"])
        
        for gate in unformatted_data['gates']:
            for param in gate['parameters']:
                qubit = gate['qubits'][0]
                gate_type = gate['gate']
                param_name = param['name']
                if param['unit'] == '':
                    param_unit = 'per'
                else:
                    param_unit = param['unit']
                param_value = param['value']
                gate_name = gate['name']
                if gate['gate'] == 'rz' or gate['gate'] == 'x':
                    continue
                if gate['gate'] != 'reset' and gate['gate'] != 'cx' and param['name'] == 'gate_length':
                    continue
                writer.writerow([qubit, gate_type, param_name, param_unit, param_value, gate_name])
    
    complete_name = os.path.join(save_path, dateString+'-general.csv')            
    with open(complete_name, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Name', 'Unit', 'Value'])
        for item in unformatted_data['general']:
            name = item['name']
            unit = item['unit']
            value = item['value']
            writer.writerow([name, unit, value])